In [1]:
import os
import time

import numpy as np
import torch
import argparse

from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer, 
                          Phi3ForCausalLM,
                          Phi3Config,
                          pipeline,
                          )
from transformers.pipelines.pt_utils import KeyDataset
import torch_tensorrt

import utils

In [2]:
parser = argparse.ArgumentParser(description="compress any prompt.")
parser.add_argument("--model_path", type=str, default="models/")
parser.add_argument("--model_name", type=str, default="Phi-3-medium-4k-instruct")
parser.add_argument("--data_path", type=str, default="data/")
parser.add_argument("--dataset", type=str, default="test_dataset.jsonl")
parser.add_argument('--seed',type=int, default=0)
parser.add_argument("--batch_size", type=int, default=1)


config = parser.parse_args([])
utils.seed_everything(config.seed)
CURR_PATH = os.getcwd()

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else cpu )
print(device)

cuda


In [4]:
model_id = os.path.join(CURR_PATH, config.model_path, config.model_name) # please replace with local model path
print(model_id)

/home/elicer/LLMInference/models/Phi-3-medium-4k-instruct


In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)

INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [7]:
my_model = model.cuda().eval()

RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.

In [12]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Tue Sep 10 11:39:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000000:65:00.0 Off |                   On |
| N/A   40C    P0              70W / 300W |                  N/A |     N/A      Default |
|                                         |                      |              Enabled |
+-----------------------------------------+----------------------+--

In [8]:
data = load_dataset("json", data_files="./data/test_dataset.jsonl")['train']

In [9]:
data[0]['message'][0]['content']

"\nRead the question and answer the following sentence in given multiple choice.\nAnswer only the sentence you chose. Never include a question and other word in your answer.\n\nquestion: Frilled sharks and angler fish live far beneath the surface of the ocean, which is why they are known as\nchoices: ['Deep sea animals', 'fish', 'Long Sea Fish', 'Far Sea Animals']\n"

In [10]:
tokenized_inputs = tokenizer(data[0]['message'][0]['content'], padding=True, truncation=True, return_tensors='pt')

In [11]:
example_input = tokenized_inputs['input_ids'].cuda()  # 첫 번째 텍스트 입력 예시
trt_model = torch_tensorrt.compile(
    my_model,
    inputs=[torch_tensorrt.Input(example_input.shape)],
    enabled_precisions={torch.float}  # FP32 사용
)

NameError: name 'my_model' is not defined

In [21]:
trt_model = torch.compile(my_model, backend="tensorrt")

In [23]:
batch_size = 4

In [46]:
data[0:0+batch_size]['message']


[[{'role': 'user',
   'content': "\nRead the question and answer the following sentence in given multiple choice.\nAnswer only the sentence you chose. Never include a question and other word in your answer.\n\nquestion: Frilled sharks and angler fish live far beneath the surface of the ocean, which is why they are known as\nchoices: ['Deep sea animals', 'fish', 'Long Sea Fish', 'Far Sea Animals']\n"}],
 [{'role': 'user',
   'content': "\nRead the question and answer the following sentence in given multiple choice.\nAnswer only the sentence you chose. Never include a question and other word in your answer.\n\nquestion: Gas can fill any container it is given, and liquid\nchoices: ['is standard weight and size', 'is the opposite of variable', 'only needs a few', 'uses what it needs']\n"}],
 [{'role': 'user',
   'content': "\nRead the question and answer the following sentence in given multiple choice.\nAnswer only the sentence you chose. Never include a question and other word in your ans

In [47]:
# 배치 처리 및 추론
for i in range(0, len(data), batch_size):
    batch_texts = []
    for j in range(0, batch_size):
        batch_texts.append(data[i:i+batch_size]['message'][j][0]['content'])
    batch_tokenized = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt')
    batch_inputs = batch_tokenized['input_ids'].cuda()

    # TensorRT 모델로 추론
    output = trt_model(batch_inputs)
    
    print(f"Batch {i // batch_size + 1} Output:\n", output)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)